In [ ]:
import os
import json
import torch
import faiss
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTModel

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained model and feature extractor
model_name = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTModel.from_pretrained(model_name).to(device)
model.eval()

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

# Load and preprocess images
def load_images_from_folder(folder_path):
    image_paths = []
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            path = os.path.join(folder_path, filename)
            try:
                image = Image.open(path).convert('RGB')
                image = preprocess(image)
                images.append(image)
                image_paths.append(path)
            except Exception as e:
                print(f"Error loading image {path}: {e}")
    return image_paths, images

gallery_folder = "Data_example/test/gallery"
query_folder   = "Data_example/test/query"

gallery_paths, gallery_images = load_images_from_folder(gallery_folder)
query_paths, query_images     = load_images_from_folder(query_folder)

# Extract embeddings
def extract_embeddings(images):
    embeddings = []
    with torch.no_grad():
        for image in images:
            image = image.unsqueeze(0).to(device)
            outputs = model(image)
            embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
            embeddings.append(embedding)
    return np.array(embeddings)

gallery_embeddings = extract_embeddings(gallery_images)
query_embeddings   = extract_embeddings(query_images)

# Normalize
gallery_embeddings = gallery_embeddings / np.linalg.norm(gallery_embeddings, axis=1, keepdims=True)
query_embeddings   = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)

# FAISS index
dimension = gallery_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(gallery_embeddings)

# Similarity search
k = 5
distances, indices = index.search(query_embeddings, k)

# Prepare submission format
results = []
for i, query_path in enumerate(query_paths):
    top_k_paths = [gallery_paths[idx] for idx in indices[i]]
    results.append({
        "filename": query_path,
        "gallery_images": top_k_paths
    })

# Save to JSON
with open("submission4.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Saved submission JSON to 'submission_vit_faiss.json'")